## Read Reanalysis
This notebooks saves pre-processed reanalysis data for later analysis

In [2]:
import numpy as np
import pandas as pd
import xarray as xr
import glob as glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [3]:
from project_utils import parameters as param

In [4]:
ncep_path = "../input_data/NCEP-NCAR-R1/"

#### 500-hPa geopotential height: 
read daily data, subtract domain average 500-hPa trend, calculate daily standardized anomalies

In [5]:
hgt_ncep_files = sorted(glob.glob(ncep_path+"hgt/*.nc"))

hgt_ds = xr.open_mfdataset(hgt_ncep_files, combine = 'nested', concat_dim='time').sel(
    lat = param.lat_bbox, 
    lon = param.lon_bbox, 
    level = param.hgt_level, 
    time = param.time_period)

lats = hgt_ds['lat']
lons = hgt_ds['lon']

In [6]:
## calculate daily area-weighted domain average 500-hPa GPH
area_weights = xr.broadcast(np.cos(np.deg2rad(lats)), hgt_ds, exclude = ['lat', 'time'])[0]
hgt_domain_mean = hgt_ds['hgt'].weighted(area_weights.lat).mean(dim = ['lat', 'lon'])

## calculate annual domain average 500-hPa GPH to remove seasonal variability 
hgt_domain_mean = hgt_domain_mean.groupby('time.year').mean(dim = "time").to_dataframe(name = "hgt")

## calculate linear trend in 500-hPa GPH
hgt_trend = np.polyfit(hgt_domain_mean['hgt'].index.get_level_values('year'), hgt_domain_mean['hgt'], 1)

print("Slope of hgt_trend: ", hgt_trend[0], "m per year")

Slope of hgt_trend:  0.5026036065599478 m per year


In [7]:
## calculate detrended hgt
hgt_ds['change'] = (hgt_ds.time.dt.year - 1981)*hgt_trend[0]
hgt_ds['hgt_detrend'] = hgt_ds['hgt'] - hgt_ds['change']
hgt_ds = hgt_ds.drop_vars('change')

In [8]:
## calculate standardized anomalies 
hgt_ds['mean'] = hgt_ds['hgt_detrend'].groupby('time.dayofyear').mean(dim = 'time')
hgt_ds['sd'] = hgt_ds['hgt_detrend'].groupby('time.dayofyear').std(dim = 'time')
hgt_ds['hgt_anom'] = (hgt_ds['hgt_detrend'].groupby('time.dayofyear') - hgt_ds['mean']).groupby('time.dayofyear')/hgt_ds['sd']

In [9]:
hgt_ds['hgt_anom'].to_netcdf("../processed_data/hgt_detrended_anomalies.nc")

#### Sea Level Pressure: 
read daily data, calculate daily standardized anomalies

In [10]:
slp_ncep_files = sorted(glob.glob(ncep_path+"slp/*.nc"))

slp_ds = xr.open_mfdataset(slp_ncep_files, combine = 'nested', concat_dim='time').sel(
    lat = param.lat_bbox, 
    lon = param.lon_bbox, 
    time = param.time_period)

In [11]:
## calculate daily standardized anomalies
slp_ds['mean'] = slp_ds['slp'].groupby('time.dayofyear').mean(dim = 'time')
slp_ds['sd'] = slp_ds['slp'].groupby('time.dayofyear').std(dim = 'time')
slp_ds['slp_anom'] = (slp_ds['slp'].groupby('time.dayofyear') - slp_ds['mean']).groupby('time.dayofyear')/slp_ds['sd']

In [12]:
slp_ds['slp_anom'].to_netcdf("../processed_data/slp_anomalies.nc")

#### Moisture Flux: 
read uwind, vwind, and shum; calculate daily vertically integrated moisture flux

In [13]:
uwnd_ncep_files = sorted(glob.glob(ncep_path+"uwnd/*.nc"))
vwnd_ncep_files = sorted(glob.glob(ncep_path+"vwnd/*.nc"))
shum_ncep_files = sorted(glob.glob(ncep_path+"shum/*.nc"))

In [14]:
u_dat = xr.open_mfdataset(uwnd_ncep_files, combine = 'nested', concat_dim='time').sel(
    lat = param.lat_bbox, lon = param.lon_bbox, time = param.time_period, 
    level = param.mflux_levels)

v_dat = xr.open_mfdataset(vwnd_ncep_files, combine = 'nested', concat_dim='time').sel(
    lat = param.lat_bbox, lon = param.lon_bbox, time = param.time_period, 
    level = param.mflux_levels)

sh_dat = xr.open_mfdataset(shum_ncep_files, combine = 'nested', concat_dim='time').sel(
    lon = param.lon_bbox, lat = param.lat_bbox, time = param.time_period, 
    level = param.mflux_levels)

**Calculate moisture flux:** 
    
$MF_x = \frac{1}{g}\int_{p_1}^{p_2}qudp$ 

$MF_y = \frac{1}{g}\int_{p_1}^{p_2}qvdp$

total moisture flux $=\sqrt{MF_x^2 + MF_y^2}$

In [17]:
moisture_flux = xr.Dataset()

# multiply by -100 because pressure was in hPa instead of Pa, 
# and pressure coordinates are decreasing but we want positive integral
moisture_flux['u_dir'] = -100*(u_dat['uwnd']*sh_dat['shum']*1/param.g).integrate(coord = 'level')
moisture_flux['v_dir'] = -100*(v_dat['vwnd']*sh_dat['shum']*1/param.g).integrate(coord = 'level')
moisture_flux['mag'] = np.sqrt((moisture_flux['u_dir']**2) + (moisture_flux['v_dir']**2))

In [21]:
moisture_flux.to_netcdf("../processed_data/moisture_flux.nc")